# Some mongodb tests

We need to figure out how we could make it work

## Create the client

first we need to import the necessary libraries

In [7]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

from bson.objectid import ObjectId
from pprint import pprint

import datetime
import uuid

In [2]:
from decouple import config

mongodb_username = config("MONGODB_USERNAME")
mongodb_password = config("MONGODB_PASSWORD")
mongodb_database_url = config("MONGODB_DATABASE_URL")

In [3]:
uri = f"mongodb+srv://{mongodb_username}:{mongodb_password}@{mongodb_database_url}/?retryWrites=true&w=majority"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi("1"))
# Send a ping to confirm a successful connection
try:
    client.admin.command("ping")
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


## Create file API

This is now the part that we would like to execute within our system.

In [4]:
def create_job_id_dropbox(backend_name: str, username: str) -> str:
    """
    Creates an appropiate job id that works with this provider
    """
    job_id = (
        (datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S"))
        + "-"
        + backend_name
        + "-"
        + username
        + "-"
        + (uuid.uuid4().hex)[:5]
    )
    return job_id


def create_job_id(backend_name: str, username: str) -> str:
    """
    Creates an appropiate job id that works with this provider
    """
    return uuid.uuid4().hex[:24]


def upload(content_dict: dict, storage_path: str, file_id: str) -> None:
    """
    Upload the file to the storage

    content_dict: the content that should be uploaded onto the mongodb base
    storage_path: the access path towards the mongodb collection
    """
    storage_splitted = storage_path.split("/")

    # get the database on which we work
    database = client[storage_splitted[0]]

    # get the collection on which we work
    collection_name = ".".join(storage_splitted[1:])
    collection = database[collection_name]

    content_dict["_id"] = ObjectId(file_id)

    result = collection.insert_one(content_dict)
    pprint(result)

get a proper job id

In [5]:
backend_name = "rydberg"
username = "fretchen"

Let's assemble the code puzzles

In [10]:
# the dictionary that we will add
content_dict = {"experiment_0": "Nothing happened here."}

# how to access the file
storage_path = "backend/result/rydberg/fred"
storage_path = "backend/result"

# the id of the job that we created from within Django
job_id = create_job_id(backend_name, username)

upload(content_dict, storage_path, job_id)

In [8]:
print(client.list_database_names())

['Backend_files', 'backend', 'library', 'mbduser_test_db', 'sample_airbnb', 'sample_analytics', 'sample_geospatial', 'sample_guides', 'sample_mflix', 'sample_restaurants', 'sample_supplies', 'sample_training', 'sample_weatherdata', 'test', 'admin', 'local']


In [9]:
def get_file_content(storage_path: str, file_id: str) -> dict:
    """
    Get the file content from the storage

    storage_path: the path towards the file, excluding the filename / id
    file_id: the id of the file we are about to look up
    """

    document_to_find = {"_id": ObjectId(file_id)}

    # get the database on which we work
    database = client[storage_path.split("/")[0]]

    # get the collection on which we work
    collection_name = ".".join(storage_path.split("/")[1:])
    collection = database[collection_name]

    result_found = collection.find_one(document_to_find)

    return result_found

In [10]:
storage_path = "backend/Result/rydberg/fred"
obtained_dict = get_file_content(storage_path, job_id)
pprint(obtained_dict)

{'_id': ObjectId('b25d88a6729f42828dc7b6eb'),
 'experiment_0': 'Nothing happened here.'}


## Move file

Try to implement the API to move the file. However, this is only really needed on the side of the sqooler

In [17]:
document_to_find = {"_id": ObjectId(job_id)}

storage_path = "backend/Result/rydberg/fred"

# get the database on which we work
database = client[storage_path.split("/")[0]]

# get the collection on which we work
collection_name = ".".join(storage_path.split("/")[1:])
collection = database[collection_name]

result_found = collection.find_one(document_to_find)

pprint(result_found)

{'_id': ObjectId('e2cf39bdbae94935963dcc6d'),
 'experiment_0': 'Nothing happened here.'}


In [ ]:
def move_file(self, start_path: str, final_path: str, file_id: str) -> None:
    """
    Move the file from start_path to `final_path`

    start_path: the access path at which we get the file (excluding the file id)
    final_path: the access path at which we put the file (excluding the file id)
    file_id: the id of the file we are about to move
    """

# Get the backends that we created for test purposes


In [13]:
# the path was set by `sqooler`
config_path = "Backend_files/Config"

# get the database on which we work
database = client[config_path.split("/")[0]]

# list all the collections in the database
pprint(database.list_collection_names())

configs = database["Config"]
# get all the documents in the collection configs and print them
for config in configs.find():
    pprint(config)

['Config']
{'_id': ObjectId('4c864623b998441fa75f5d65'),
 'gates': [],
 'name': 'dummy_4c864623b998441fa75f5d65',
 'num_wires': 3,
 'version': '0.0.1'}
{'_id': ObjectId('92d53c2ec63b43b2853fd004'),
 'display_name': 'dummy_92d53c2ec63b43b2853fd004',
 'gates': [],
 'name': 'Dummy',
 'num_wires': 3,
 'version': '0.0.1'}


In [14]:
requested_backend = "dummy_92d53c2ec63b43b2853fd004"

document_to_find = {"display_name": requested_backend}

config = configs.find_one(document_to_find)
pprint(config)

{'_id': ObjectId('92d53c2ec63b43b2853fd004'),
 'display_name': 'dummy_92d53c2ec63b43b2853fd004',
 'gates': [],
 'name': 'Dummy',
 'num_wires': 3,
 'version': '0.0.1'}
